In [ ]:
import os
import sys

In [ ]:
os.path.abspath(os.path.join('../'))

# Importing required Libraries

In [ ]:
# Data handling
import pandas as pd
import numpy as np

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Preprocessing & Models
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score

# Hierarchical Clustering
from scipy.cluster.hierarchy import dendrogram, linkage

# Similarity Measures for Recommendation
from sklearn.metrics.pairwise import cosine_similarity

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Loading the Dataset

In [ ]:
# Load the dataset
df = pd.read_csv('../data/raw/online_retail.csv', encoding = 'latin1')
df

# Dataset overview

In [ ]:
# Checking the shape of the dataset
df.shape

In [ ]:
# checking the first few rows of the dataset
df.head(10)

In [ ]:
# checking the last few rows of the dataset
df.tail(10)

In [ ]:
# checking the info of the dataset
df.info()

In [ ]:
# displaying basic statistics of the dataset
df.describe()

# Data Cleaning

In [ ]:
# Before cleaning the data i will create a copy of the original dataframe
df_clean = df.copy()

In [ ]:
# checking for missing values
df_clean.isnull().sum()

In [ ]:
# removing the rows with missing CustomerID
df_clean = df_clean.dropna(subset=["CustomerID"])

# Filling out the missing values in 'Description' with 'Unknown'
df_clean['Description'] = df_clean['Description'].fillna('Unknown')

In [ ]:
# checking the data after handling missing values
df_clean.isnull().sum()

In [ ]:
print(df_clean.columns.to_list())

In [ ]:
# Removing the cancelled invoices number
df_clean = df_clean[~df_clean['InvoiceNo'].astype(str).str.startswith('c', na=False)]

In [ ]:
# Removing the invalid quantities and prices
df_clean = df_clean[(df_clean["Quantity"] > 0) & (df_clean["UnitPrice"] > 0)]

In [ ]:
# Comparing the shape of the cleaned dataset with the original dataset
print(f"Cleaned dataset shape: {df_clean.shape}")
print(f"Original dataset shape: {df.shape}")

# Feature Engineering

In [ ]:
# Converting 'InvoiceDate' to datetime format
df_clean["InvoiceDate"] = pd.to_datetime(df_clean["InvoiceDate"])

# Creating a new column 'TotalAmount' as Quantity * UnitPrice
df_clean["TotalAmount"] = df_clean["Quantity"] * df_clean["UnitPrice"]

df_clean.head()

# Exploratory Data Analysis

## Transactions by Country

In [ ]:
# Trabsactions by Country

ax = df_clean["Country"].value_counts().head(10).plot(
    kind='bar', figsize=(12, 5), color='skyblue'
)

for c in ax.containers:
    ax.bar_label(c)

plt.title("Top 10 Countries by Number of Transactions")
plt.ylabel("Transactions")
plt.show()

## Top Selling Products

In [ ]:
# Top Selling Products
ax = df_clean.groupby("Description")["Quantity"].sum().sort_values(ascending=False).head(10).plot(
    kind = "bar", figsize=(12,5), color='salmon'
)

for c in ax.containers:
    ax.bar_label(c)


plt.title("Top 10 Selling Products")
plt.ylabel("Total Quantity Sold")
plt.show()

## Monthly Purchase trends

In [ ]:
# Monthly Purchase trends

ax = (
    df_clean.set_index("InvoiceDate").resample("M")["InvoiceNo"].nunique().plot(
        figsize=(12,5), marker='o', color='purple'
    )
)

plt.title("Monthly Transaction Trend")
plt.ylabel("Number of Transactions")
plt.xlabel("Month")
plt.grid(True, linestyle='--', alpha=0.6)

plt.show()

## Monetary per Transaction Distribution

In [ ]:
# Monetary per Transaction Distribution

df_clean["TotalPrice"] = df_clean["Quantity"] * df_clean["UnitPrice"]

plt.figure(figsize=(12,5))
plt.hist(df_clean["TotalPrice"], bins=50)
plt.title("Monetary Distribution per Transaction")
plt.xlabel("Transaction Value")
plt.ylabel("Frequency")
plt.show()


## Monetary per Customer Distribution

In [ ]:
# Monetary per Customer Distribution

customer_monetary = df_clean.groupby("CustomerID")["TotalPrice"].sum()

plt.figure(figsize=(12,5))
plt.hist(customer_monetary, bins=50)
plt.title("Monetary Distribution per Customer")
plt.xlabel("Total Spend per Customer")
plt.ylabel("Number of Customers")
plt.show()

## Creating a RFM Table

In [ ]:
# Creating RFM Table

snapshot_date = df_clean["InvoiceDate"].max() + pd.Timedelta(days=1)

rfm = df_clean.groupby("CustomerID").agg({
    "InvoiceDate": lambda x: (snapshot_date - x.max()).days,
    "InvoiceNo": "nunique",
    "TotalPrice": "sum"
})

rfm.columns = ["Recency", "Frequency", "Monetary"]
rfm.head()

## Visualizing RFM Distributions

In [ ]:
# Visualizing RFM Distributions

rfm.hist(figsize=(12,4), bins=30)
plt.tight_layout()
plt.show()

In [ ]:
# Scaling RFM Values

scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm)

In [ ]:
# Elbow Method to find optimal K

inertia = []

for k in range(1, 11):
    km = KMeans(n_clusters=k, random_state=42)
    km.fit(rfm_scaled)
    inertia.append(km.inertia_)

plt.figure(figsize=(12,5))
plt.plot(range(1, 11), inertia, marker='o')
plt.title("Elbow Curve for Optimal Clusters")
plt.xlabel("Number of Clusters")
plt.ylabel("Inertia")
plt.show()

In [ ]:
# fitting KMeans with optimal clusters
kmeans = KMeans(n_clusters=4, random_state=42)
rfm['Cluster'] = kmeans.fit_predict(rfm_scaled)

In [ ]:
# Cluster Profile Table
cluster_profile = rfm.groupby("Cluster").mean()
cluster_profile

In [ ]:
# Visualizing Cluster Comparison

cluster_profile.plot(kind="bar", figsize=(12,5))
plt.title("Customer Cluster Profiles (RFM)")
plt.ylabel("Average Value")
plt.show()


## Product Recommendation Heatmap / Similarity matrix

In [ ]:
# Creating Customer-Product Matrix

customer_product = df_clean.pivot_table(
    index="CustomerID",
    columns="Description",
    values="Quantity",
    #aggfunc="sum",
    fill_value=0
)
#customer_product.head()

In [ ]:
# Product-product Similarity Matrix

product_similarity = cosine_similarity(customer_product.T)
similarity_df = pd.DataFrame(
    product_similarity,
    index=customer_product.columns,
    columns=customer_product.columns
)
#similarity_df.head()

In [ ]:
product_name = similarity_df.columns[0]

similarity_df[product_name].sort_values(ascending=False)[1:6]

In [ ]:
# Heatmaps of Top Products

top_products = df_clean["Description"].value_counts().head(10).index
top_similarity = similarity_df.loc[top_products, top_products]

plt.figure(figsize=(10,6))
plt.imshow(top_similarity)
plt.colorbar()
plt.xticks(range(len(top_products)), top_products, rotation=90)
plt.yticks(range(len(top_products)), top_products)
plt.title("Product Similarity Heatmap")
plt.show()

In [ ]:
dbscan = DBSCAN(eps=0.8, min_samples=5)
rfm["DBSCAN_Cluster"] = dbscan.fit_predict(rfm_scaled)

rfm["DBSCAN_Cluster"].value_counts()

In [ ]:
linked = linkage(rfm_scaled, method="ward")

plt.figure(figsize=(12,5))
dendrogram(linked)
plt.title("Hierarchical Clustering Dendrogram")
plt.show()


In [ ]:
rfm.groupby("Cluster")[["Recency", "Frequency", "Monetary"]].mean()

In [ ]:
#from sklearn.metrics import silhouette_score

# KMeans Silhouette Score
kmeans_score = silhouette_score(rfm_scaled, rfm["Cluster"])

# DBSCAN Silhouette Score
dbscan_labels = rfm["DBSCAN_Cluster"]

if len(set(dbscan_labels)) > 1:
    dbscan_score = silhouette_score(rfm_scaled, dbscan_labels)
else:
    dbscan_score = "Not Applicable"

print("Silhouette Scores")
print("KMeans :", kmeans_score)
print("DBSCAN :", dbscan_score)
